
# Generate a QA Evaluation Dataset from ChromaDB (10% sample) for RAG Testing

**Audience:** 4th‑year CS students  
**Goal:** Build an **evaluation dataset** for RAG by sampling **10%** of your **pre‑chunked ChromaDB** collection and generating up to **5 question–answer pairs per chunk** using **OpenAI** *or* a **Hugging Face** model. Save the result to **CSV**.

### Why a relatively powerful LLM for question generation?
- **Answerability & grounding:** It must read a chunk and ask **non‑trivial but answerable** questions grounded **only** in that chunk (no outside knowledge).  
- **Paraphrase variety:** Better models generate diverse phrasings, reducing evaluation overfitting.  
- **Domain nuance:** Strong models handle **technical jargon** and produce **faithful** answers rather than hallucinations.  
- **Format adherence:** Higher‑end models more reliably output **valid JSON** you can parse automatically.

> This notebook keeps costs in check by sampling **10%** of chunks and capping to **≤5 Q/A per chunk**.



## 0) Setup & Installation

Uncomment the cell below if you need to install dependencies.  
We support **either** OpenAI **or** Hugging Face for question generation.


In [ ]:

# !pip install -U chromadb openai transformers accelerate torch pandas numpy tqdm python-dotenv
# If you're on Apple Silicon:
# !pip install 'torch>=2.2.0' --index-url https://download.pytorch.org/whl/cpu



### Configuration
- **ChromaDB:** point to your persisted DB folder and collection name.  
- **Sampling:** set `SAMPLE_FRACTION=0.10` (10%).  
- **Generation choice:** set `USE_OPENAI=True` (needs `OPENAI_API_KEY`) or `USE_HF=True` (downloads HF model).  
- **Safety knobs:** `MAX_Q_PER_CHUNK` (≤5), `MAX_CHARS_PER_CHUNK` to truncate very long chunks.

> If both `USE_OPENAI` and `USE_HF` are True, OpenAI is used by default.


In [ ]:

import os
import json
import random
from typing import List, Dict, Any
from dataclasses import dataclass

# ---- User-editable parameters ----
PERSIST_DIRECTORY = os.getenv("CHROMA_PERSIST_DIR", "./chroma")
COLLECTION_NAME   = os.getenv("CHROMA_COLLECTION", "my_chunks")

SAMPLE_FRACTION   = float(os.getenv("SAMPLE_FRACTION", "0.10"))  # 10% of chunks
MAX_Q_PER_CHUNK   = int(os.getenv("MAX_Q_PER_CHUNK", "5"))       # cap at 5
MAX_CHARS_PER_CHUNK = int(os.getenv("MAX_CHARS_PER_CHUNK", "3000"))
RANDOM_SEED       = int(os.getenv("RANDOM_SEED", "471"))

# Generation backends
USE_OPENAI = os.getenv("USE_OPENAI", "False").lower() == "true"
USE_HF     = os.getenv("USE_HF", "True").lower() == "true"  # default to HF to avoid API keys

# OpenAI config
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # set in env or .env

# Hugging Face config (choose a small instruct model if running on CPU)
HF_TASK  = "text2text-generation"  # for instruction-style prompting
HF_MODEL = os.getenv("HF_MODEL", "google/flan-t5-base")  # lightweight; upgrade if you have GPU
HF_MAX_NEW_TOKENS = int(os.getenv("HF_MAX_NEW_TOKENS", "512"))
HF_DO_SAMPLE = os.getenv("HF_DO_SAMPLE", "False").lower() == "true"

# Output
OUTPUT_CSV = os.getenv("OUTPUT_CSV", "rag_eval_dataset.csv")

random.seed(RANDOM_SEED)



## 1) Connect to ChromaDB and load chunks

We assume your data is **already chunked** and stored in a Chroma collection.  
We’ll read IDs, documents, and metadatas in **pages** to avoid loading everything at once.


In [ ]:

import chromadb

client = chromadb.PersistentClient(path=PERSIST_DIRECTORY)
collection = client.get_collection(COLLECTION_NAME)

total = collection.count()
print(f"Total chunks in collection '{COLLECTION_NAME}': {total}")


In [ ]:

# ---- Retrieve all ids, docs, and metadatas with paging ----
ALL_IDS, ALL_DOCS, ALL_META = [], [], []

PAGE = 1000
offset = 0
while True:
    batch = collection.get(
        include=["documents", "metadatas"],
        limit=PAGE,
        offset=offset
    )
    ids = batch.get("ids", [])
    docs = batch.get("documents", [])
    metas = batch.get("metadatas", [])
    if not ids:
        break
    ALL_IDS.extend(ids)
    ALL_DOCS.extend(docs)
    ALL_META.extend(metas)
    offset += len(ids)
    if offset >= total:
        break

print(f"Loaded {len(ALL_IDS)} items from Chroma.")
assert len(ALL_IDS) == len(ALL_DOCS) == len(ALL_META)



## 2) Sample ~10% of chunks (reproducible)

We take a 10% sample to keep cost/time manageable. You can adjust `SAMPLE_FRACTION`.


In [ ]:

n_total = len(ALL_IDS)
n_sample = max(1, int(n_total * SAMPLE_FRACTION))
indices = list(range(n_total))
random.shuffle(indices)
sample_idx = sorted(indices[:n_sample])

print(f"Sampling {n_sample} / {n_total} chunks (~{SAMPLE_FRACTION*100:.1f}%).")

SAMPLED = [
    {
        "chunk_id": ALL_IDS[i],
        "text": (ALL_DOCS[i] or "")[:MAX_CHARS_PER_CHUNK],  # truncate long chunks
        "metadata": ALL_META[i] or {}
    }
    for i in sample_idx
]
print(f"Prepared {len(SAMPLED)} chunks for question generation.")



## 3) Prompt design for grounded Q/A

We ask the model to produce **up to 5 Q/A pairs** that are **fully answerable from the chunk** and return **valid JSON**:

```json
[
  {"question": "...", "answer": "..."},
  ...
]
```
If the chunk lacks enough information, the model should return **[]**.


In [ ]:

JSON_INSTRUCTIONS = (
    "Return ONLY a JSON array of objects, each with keys 'question' and 'answer'. "
    "Do not include any extra commentary. If the chunk lacks enough info, return []."
)

def build_prompt(chunk_text: str, max_q: int) -> str:
    return (
        "You are a careful question writer for Retrieval-Augmented Generation (RAG).\n"
        "Read the CHUNK below and create up to {max_q} question–answer pairs that are:\n"
        " - Non-trivial but fully answerable using ONLY the CHUNK\n"
        " - Concise, precise, and faithful to the CHUNK (no outside knowledge)\n"
        " - Useful for evaluating a retriever's ability to find this CHUNK\n\n"
        f"{JSON_INSTRUCTIONS}\n\n"
        "CHUNK:\n"
        f"{chunk_text}\n\n"
        "JSON:"
    ).format(max_q=max_q)



## 4) Choose a generation backend

Set **`USE_OPENAI=True`** to call OpenAI Chat Completions; otherwise we default to a **Hugging Face** model (`text2text-generation`).  
For CPU‑only environments, start with `google/flan-t5-base` (lower quality, but free). On GPU, try a stronger instruct model.


In [ ]:

# ---- OpenAI backend (chat completions) ----
def openai_generate_json(prompt: str) -> str:
    """Return raw JSON string from OpenAI Chat Completions."""
    try:
        from openai import OpenAI
        client = OpenAI(api_key=OPENAI_API_KEY)
        resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role":"system","content":"You write grounded Q/A pairs in strict JSON."},
                {"role":"user","content": prompt}
            ],
            temperature=0.2
        )
        return resp.choices[0].message.content
    except Exception as e:
        raise RuntimeError(f"OpenAI generation failed: {e}")


In [ ]:

# ---- Hugging Face backend (text2text-generation) ----
from transformers import pipeline

_hf_pipe = None
def hf_generate_json(prompt: str) -> str:
    global _hf_pipe
    if _hf_pipe is None:
        _hf_pipe = pipeline(HF_TASK, model=HF_MODEL)
    out = _hf_pipe(
        prompt,
        max_new_tokens=HF_MAX_NEW_TOKENS,
        do_sample=HF_DO_SAMPLE
    )
    # HF pipelines return a list of dicts; try the first item
    text = out[0].get("generated_text", "")
    return text



## 5) Robust JSON parsing of model output

Models sometimes wrap JSON in prose or code fences. We defensively extract the first JSON array.
If parsing fails, we fall back to a very simple regex that looks for `"question"` and `"answer"` pairs.


In [ ]:

import re
import json
from typing import List, Dict, Any

def extract_json_array(text: str) -> List[Dict[str, Any]]:
    # Try to find a JSON array block
    # 1) Inline code fence
    fence = re.findall(r"```(?:json)?\s*(\[.*?\])\s*```", text, flags=re.S)
    candidates = fence if fence else re.findall(r"(\[\s*{.*?}\s*\])", text, flags=re.S)
    if candidates:
        raw = candidates[0]
    else:
        raw = text.strip()
    try:
        arr = json.loads(raw)
        if isinstance(arr, list):
            # sanitize items
            clean = []
            for it in arr:
                if isinstance(it, dict) and 'question' in it and 'answer' in it:
                    q = str(it['question']).strip()
                    a = str(it['answer']).strip()
                    if q and a:
                        clean.append({'question': q, 'answer': a})
            return clean
        return []
    except Exception:
        # Fallback: scrape "question":"...", "answer":"..." naive pairs
        pairs = []
        pattern = re.findall(r'"question"\s*:\s*"(.*?)"\s*,\s*"answer"\s*:\s*"(.*?)"', text, flags=re.S)
        for q,a in pattern:
            q = q.strip().replace('\n',' ')
            a = a.strip().replace('\n',' ')
            if q and a:
                pairs.append({'question': q, 'answer': a})
        return pairs



## 6) Generate Q/A pairs for the sampled chunks

We iterate over the sample, build a grounded prompt, call the selected backend, parse JSON,
and collect **up to 5 Q/A** per chunk.


In [ ]:

from tqdm import tqdm
import pandas as pd

backend = "openai" if USE_OPENAI else "hf"
print(f"Using backend: {backend}")

rows = []
for item in tqdm(SAMPLED, desc="Generating Q/A"):
    chunk_id = item["chunk_id"]
    text = item["text"] or ""
    meta = item["metadata"] or {}

    if not text.strip():
        continue

    prompt = build_prompt(text, MAX_Q_PER_CHUNK)
    try:
        if USE_OPENAI:  # prefer OpenAI if explicitly enabled
            raw = openai_generate_json(prompt)
        else:
            raw = hf_generate_json(prompt)
        qa_list = extract_json_array(raw)[:MAX_Q_PER_CHUNK]
    except Exception as e:
        qa_list = []
        print(f"[WARN] Generation failed for chunk {chunk_id}: {e}")

    preview = (text[:220] + "…") if len(text) > 220 else text
    source = meta.get("source") or meta.get("file_path") or meta.get("url") or ""

    for qa in qa_list:
        rows.append({
            "chunk_id": chunk_id,
            "question": qa["question"],
            "answer": qa["answer"],
            "source": source,
            "metadata": json.dumps(meta, ensure_ascii=False),
            "chunk_preview": preview
        })

df = pd.DataFrame(rows, columns=["chunk_id","question","answer","source","metadata","chunk_preview"])
print(f"Generated {len(df)} Q/A rows.")
df.head(5)



## 7) Save evaluation dataset to CSV

This CSV can be loaded by your **RAG evaluation** notebook to compute **Recall@K, Precision@K, MAP, and MRR**.


In [ ]:

df.to_csv(OUTPUT_CSV, index=False)
print(f"Saved evaluation dataset to: {OUTPUT_CSV}")



## 8) Tips & Next Steps

- **Quality control:** Spot‑check a few rows to ensure questions are **answerable from the chunk** and answers are **faithful**.  
- **Stronger models ⇒ better Q/A:** If using HF on CPU (e.g., `flan‑t5‑base`), consider upgrading to a stronger instruct model on GPU for higher‑quality questions.  
- **De‑duplication:** Remove near‑duplicate questions across chunks to reduce evaluation bias.  
- **Balance sampling:** You can stratify sampling by source/file/topic to ensure coverage.  
- **Costs:** OpenAI usage scales with tokens; keep chunks truncated and sample fraction small to control spend.
